In [ ]:
!pip install beeai.framework
!pip install beeai


  Using cached beeai-1.0.1-py3-none-any.whl.metadata (627 bytes)
  Using cached python_socketio-5.12.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached bidict-0.23.1-py3-none-any.whl.metadata (8.7 kB)
  Using cached python_engineio-4.11.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached simple_websocket-1.1.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached beeai-1.0.1-py3-none-any.whl (4.0 kB)
Using cached python_socketio-5.12.1-py3-none-any.whl (76 kB)
Using cached bidict-0.23.1-py3-none-any.whl (32 kB)
Using cached python_engineio-4.11.2-py3-none-any.whl (59 kB)
Using cached simple_websocket-1.1.0-py3-none-any.whl (13 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)


In [ ]:
!pip install ibm_watsonx_ai


In [ ]:
!pip install --upgrade beeai-framework

In [ ]:
import os

from beeai_framework.adapters.watsonx.backend.chat import WatsonxChatModel

WATSONX_API_KEY = "YOUR_API_KEY"
WATSONX_PROJECT_ID = "YOUR_PROJECT_ID"
WATSONX_API_URL =  "https://us-south.ml.cloud.ibm.com"

os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID

os.environ["WATSONX_API_KEY"] = WATSONX_API_KEY

os.environ["WATSONX_API_BASE"] = WATSONX_API_URL


llm = WatsonxChatModel(
"ibm/granite-3-2b-instruct",
settings={
	"project_id":WATSONX_PROJECT_ID,
	"api_key": WATSONX_API_KEY,
	"url":WATSONX_API_URL
})

In [ ]:
!pip install beeai_framework

In [ ]:

from beeai_framework.workflows.agent import AgentWorkflow, AgentWorkflowInput

import asyncio

workflow = AgentWorkflow(name="Project Manager Assistant")

workflow.add_agent(
    name="project planner agent",
    role='Project Planner from project description',
    instructions=f"""
        You are an expert tasked with analyzing the project description:
        Your objectives are to:
        1. Extract Actionable Tasks:
            - Identify and list all actionable and realistic tasks necessary to complete the project.
            - Provide an estimated number of days required to complete each task.
        2. Refine Long-Term Tasks:
            - For any task estimated to take longer than 5 days, break it down into smaller, independent sub-tasks.
        Requirements: - Ensure each task is clearly defined and achievable.
            - Maintain logical sequencing of tasks to facilitate smooth project execution.""",
    llm=llm
)

workflow.add_agent(
    name="resource allocator agent",
    role='Assignment of tasks to employees',
    instructions=f"""
        You are an expert responsible for allocating tasks to team members efficiently.
        Your objectives are to:
            1. Allocate Tasks:
                - Assign each task to a team member based on their expertise and current availability.
                - Ensure that no team member is assigned overlapping tasks during the same time period.
            2. Optimize Assignments:
                - Utilize insights from previous iterations to improve task allocations.
                - Balance the workload evenly among team members to enhance productivity and prevent burnout.
                Constraints:
                    - Each team member can handle only one task at a time.
                    - Assignments should respect the skills and experience of each team member.
        """,
    llm=llm)

workflow.add_agent(
    name="timeline generator agent",
    role='Setting the timeline of tasks',
    instructions=f"""
        You are an experienced project scheduler tasked with creating an optimized project timeline.
            1. Develop a Task Schedule:
                - Assign start and end days to each task, ensuring that all dependencies are respected.
                - Optimize the schedule to minimize the overall project duration.
                - If possible parallelize the tasks to reduce the overall project duration.
                - Try not to increase the project duration compared to previous iterations.
                - Ensure that you give the start and end dates in the format yyyy-mm-dd
        """,
    llm=llm)

workflow.add_agent(
    name="summary generator",
    role="Generate summary of the entire project plan into summary",
    instructions="""You are an expert in generating summary about the project plan including the tasks, resource allocation and timeline details of a project in a structured format with a list of tasks dictionaries containing Task, Start, End and Resource for each task""",
    llm=llm
)

workflow.add_agent(
    name="gantt chart generator",
    role="Generate and visualize project timelines by converting the structured task schedule into a Gantt chart visualisation",
    instructions="""You are a data visualization expert who generates python scripts for gantt charts
    Ensure that you output the python code as a function called gantt_chart_generator
    Take the below code as reference:
    import plotly.express as px
    import pandas as pd

    df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
    ])

    fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Resource")
    fig.update_yaxes(autorange="reversed")
    fig.show()""",
    llm=llm
)

result =  await workflow.run(inputs=[
    AgentWorkflowInput(prompt="Office Building construction project.",
    expected_output="A list of tasks based on the project description."),

    AgentWorkflowInput(prompt="""Employees: Alice - Project Manager, Bob - Architect, Charlie - Civil Engineer, Diana - Site Supervisor, Eve - Electrician, Frank - Plumber, Grace - Safety Officer, Henry - Procurement Manager. Assign the identified tasks to the these employees based on their roles and expertise.""",
                       expected_output="Assign the tasks to employees and give the information."),

    AgentWorkflowInput(prompt="Project duration is is starting from 2025-03-01 and ending at 2025-07-01. Decide and optimize the timeline for the tasks and assign start and end dates (yyyy-mm-dd)",
                       expected_output="Decide and optimize the timeline based on the project duration in the format yyyy-mm-dd."),

    AgentWorkflowInput(prompt="Generate a summary of the entire project plan including tasks, resource allocations, and timelines.",
                       expected_output="Generate accurate summary of the entire project plan based on tasks, resource allocation and timelines of the project. The format of the output should be tasks(list of dict): Each dict should have Task, Start, Finish and Resource."),

    AgentWorkflowInput(prompt="Generate Python code for a function named 'gantt_chart_generator' that creates a Gantt chart from the summary data.",
                       expected_output="Generate python script to generate gantt chart with only the function gantt_chart_generator and parse the dates in %Y-%m-%d format using pd.DateTime(). Make sure to call the function.")

])

print(result)


state=Schema(inputs=[], final_answer='```python\nimport plotly.express as px\nimport pandas as pd\n\n# Summary data as a list of dictionaries\nsummary_data = [\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-01",\n        "Finish": "2025-03-03",\n        "Resource": "Alice"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-02",\n        "Finish": "2025-03-02",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-03",\n        "Finish": "2025-03-03",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-04",\n        "Finish": "2025-03-10",\n        "Resource": "Bob"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-11",\n        "Finish": "2025-03-11",\n        "Resource": "Charlie"\n    },\n    {\n        "Task": "Material Procurement",\n        "Start": "2025-03-12",\n        "Finish":

In [ ]:
output = list(result)
code = output[0][1]
code

Full output:
inputs=[] final_answer='```python\nimport plotly.express as px\nimport pandas as pd\n\n# Summary data as a list of dictionaries\nsummary_data = [\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-01",\n        "Finish": "2025-03-03",\n        "Resource": "Alice"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-02",\n        "Finish": "2025-03-02",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-03",\n        "Finish": "2025-03-03",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-04",\n        "Finish": "2025-03-10",\n        "Resource": "Bob"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-11",\n        "Finish": "2025-03-11",\n        "Resource": "Charlie"\n    },\n    {\n        "Task": "Material Procurement",\n        "Start": "2025-03-12",\n        "Finish": 

In [ ]:
# Extracting Python code
import re
xyz = code.final_answer
print(xyz)
match = re.search(r"```python\n(.*?)\n```", xyz, re.DOTALL)
if match:
    python_code = match.group(1)
    print(python_code)
    exec(python_code)
else:
    print("No Python code found.")

Raw code string representation:
'```python\nimport plotly.express as px\nimport pandas as pd\n\n# Summary data as a list of dictionaries\nsummary_data = [\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-01",\n        "Finish": "2025-03-03",\n        "Resource": "Alice"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-02",\n        "Finish": "2025-03-02",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-03",\n        "Finish": "2025-03-03",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-04",\n        "Finish": "2025-03-10",\n        "Resource": "Bob"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-11",\n        "Finish": "2025-03-11",\n        "Resource": "Charlie"\n    },\n    {\n        "Task": "Material Procurement",\n        "Start": "2025-03-12",\n        "Finish": "202

SyntaxError: unterminated string literal (detected at line 117) (<string>, line 117)

Full output:
inputs=[] final_answer='```python\nimport plotly.express as px\nimport pandas as pd\n\n# Summary data as a list of dictionaries\nsummary_data = [\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-01",\n        "Finish": "2025-03-03",\n        "Resource": "Alice"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-02",\n        "Finish": "2025-03-02",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Site Preparation",\n        "Start": "2025-03-03",\n        "Finish": "2025-03-03",\n        "Resource": "Diana"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-04",\n        "Finish": "2025-03-10",\n        "Resource": "Bob"\n    },\n    {\n        "Task": "Design and Engineering",\n        "Start": "2025-03-11",\n        "Finish": "2025-03-11",\n        "Resource": "Charlie"\n    },\n    {\n        "Task": "Material Procurement",\n        "Start": "2025-03-12",\n        "Finish": 

SyntaxError: unterminated string literal (detected at line 117) (<string>, line 117)